# Modelación del territorio utilizando PCA

En este cuaderno llevamos a cabo una modelización de un territorio, en este caso toda España, mediante PCA en las elecciones de noviembre de 2019. 

En el cuaderno anterior nos basamos en las secciones elegidas de la provincia de Zaragoza para modelizar esta provincia. Ahora seleccionaremos las secciones a partir de una corta serie de municipios; es decir, estaremos siendo bastante restrictivos desde el principio. Algunos municipios estarán situados en CCAA con partidos regionalistas o nacionalistas, para intentar captar las particuparidades del voto en estos territorios. 

El método hasta aplicar la PCA es exactamente el mismo que el que utilizamos en el cuaderno de regresión lineal, por lo que no nos detendremos en nuestros comentarios tan detalladamente como en aquel. También aplicaremos el modelo obtenido para ver lo bien que es capaz de predecir, con las secciones equivalentes en junio de 2016, los resultados de esa elección.

Comenzamos cargando las librerías y el dataset de noviembre de 2019.

In [146]:
import pandas as pd
import numpy as np
import random

In [147]:
strings = {'Sección' : 'str', 'cod_ccaa' : 'str', 'cod_prov' : 'str', 'cod_mun' : 'str', 'cod_sec' : 'str'}

In [148]:
df_eleccion_comp = pd.read_csv('/content/drive/MyDrive/Proyecto_KeepCoding - Propio/Data/Gen-19-Nov/gen_N19_unif_cols_prov.txt', dtype = strings)

In [149]:
df_eleccion_comp

,Sección,cod_ccaa,cod_prov,cod_mun,cod_sec,CCAA,Provincia,Municipio,Censo_Esc,Votos_Total,Participación,Nulos,Votos_Válidos,Blanco,V_Cand,PP,PSOE,Cs,UP,IU,VOX,UPyD,MP,CiU,ERC,JxC,CUP,DiL,PNV,Bildu,Amaiur,CC,FA,TE,BNG,PRC,GBai,Compromis,PACMA,Otros,...,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90-94,95-99,100 y más,Población Total,Hombres,Mujeres,% mayores 65 años,% 20-64 años,% menores 19 años,Afiliados SS Minicipio,% Afiliados SS autónomos,% Afiliados SS / Población,Paro Registrado Municipio,% Paro Hombres,% Paro mayores 45,% Paro s/ Afiliados SS Municipio,Renta persona 2017,Renta persona 2015,Renta hogar 2017,Renta hogar 2015,Renta Salarios 2018,Renta Salarios 2015,Renta Pensiones 2018,Renta Pensiones 2015,Renta Desempleo 2018,Renta Desempleo 2015,dict_res,dict_res_ord
0,022019111010400101001,01,04,04001,0400101001,Andalucía,Almería,Abla,1002,717,0.715569,7,710,3,707,193,310,47,30,0,122,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,2,...,73.0,80.0,89.0,81.0,94.0,87.0,91.0,77.0,72.0,42.0,67.0,56.0,19.0,4.0,0.0,1249.0,635.0,614.0,0.269816,0.590072,0.140112,304.0,0.223684,0.243395,140.0,0.421429,0.550000,0.315315,9159.0,8788.0,20172.0,19546.0,5574.0,4833.0,3286.0,3082.0,403.0,471.0,"{'PP': 193, 'PSOE': 310, 'Cs': 47, 'UP': 30, '...","[('PSOE', 310), ('PP', 193), ('VOX', 122), ('C..."
1,022019111010400201001,01,04,04002,0400201001,Andalucía,Almería,Abrucena,1013,711,0.701876,12,699,1,698,111,349,45,42,0,147,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,...,60.0,75.0,70.0,70.0,108.0,101.0,99.0,86.0,61.0,64.0,61.0,46.0,14.0,2.0,1.0,1202.0,637.0,565.0,0.278702,0.609817,0.111481,298.0,0.251678,0.247920,179.0,0.379888,0.625698,0.375262,8827.0,8107.0,17841.0,17115.0,4640.0,4048.0,3418.0,2770.0,568.0,620.0,"{'PP': 111, 'PSOE': 349, 'Cs': 45, 'UP': 42, '...","[('PSOE', 349), ('VOX', 147), ('PP', 111), ('C..."
2,022019111010400301001,01,04,04003,0400301001,Andalucía,Almería,Adra,667,484,0.725637,7,477,5,472,176,128,15,34,0,116,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,...,54.0,54.0,54.0,61.0,82.0,75.0,67.0,48.0,37.0,40.0,26.0,15.0,3.0,1.0,0.0,892.0,435.0,457.0,0.190583,0.643498,0.165919,7968.0,0.382530,8.932735,2525.0,0.432871,0.473663,0.240637,8965.0,8267.0,26498.0,24688.0,5121.0,4795.0,2499.0,2301.0,337.0,333.0,"{'PP': 176, 'PSOE': 128, 'Cs': 15, 'UP': 34, '...","[('PP', 176), ('PSOE', 128), ('VOX', 116), ('U..."
3,022019111010400301002,01,04,04003,0400301002,Andalucía,Almería,Adra,1306,909,0.696018,3,906,5,901,251,220,51,58,0,312,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,3,...,108.0,158.0,162.0,150.0,140.0,119.0,103.0,67.0,49.0,37.0,30.0,14.0,7.0,1.0,0.0,1752.0,865.0,887.0,0.117009,0.647260,0.235731,7968.0,0.382530,4.547945,2525.0,0.432871,0.473663,0.240637,8599.0,7941.0,25677.0,23400.0,5381.0,4837.0,1815.0,1724.0,343.0,464.0,"{'PP': 251, 'PSOE': 220, 'Cs': 51, 'UP': 58, '...","[('VOX', 312), ('PP', 251), ('PSOE', 220), ('U..."
4,022019111010400301003,01,04,04003,0400301003,Andalucía,Almería,Adra,1551,975,0.628627,12,963,9,954,292,202,73,52,0,327,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,3,...,189.0,178.0,215.0,227.0,164.0,110.0,96.0,61.0,58.0,41.0,40.0,27.0,4.0,4.0,0.0,2240.0,1094.0,1146.0,0.104911,0.647768,0.247321,7968.0,0.382530,3.557143,2525.0,0.432871,0.473663,0.240637,8076.0,7150.0,22051.0,19687.0,5224.0,4044.0,1170.0,1198.0,416.0,476.0,"{'PP': 292, 'PSOE': 202, 'Cs': 73, 'UP': 52, '...","[('VOX', 327), ('PP', 292), ('PSOE', 202), ('C..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36297,022019111195200108011,19,52,52001,5200108011,Melilla,Melilla,Melilla,1638,1021,0.623321,3,1018,11,1007,303,140,30,28,0,158,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,348,...,181.0,185.0,171.0,164.0,165.0,180.0,155.0,97.0,38.0,34.0,19.0,16.0,4.0,3.0,0.0,2480.0,1244.0,1236.0,0.0

Seleccionamos las secciones que vamos a modelizar, que son las de toda España.

In [150]:
ccaa_mod = []

provincia_mod = []

municipio_mod = []

secciones_mod = df_eleccion_comp

In [151]:
if len(ccaa_mod) > 0:

  secciones_mod = secciones_mod.loc[secciones_mod['CCAA'].isin(ccaa_mod)]

if len(provincia_mod) > 0:

  secciones_mod = secciones_mod.loc[secciones_mod['Provincia'].isin(provincia_mod)]

if len(municipio_mod) > 0:

  secciones_mod = secciones_mod.loc[secciones_mod['Municipio'].isin(municipio_mod)]



In [152]:
secciones_mod

,Sección,cod_ccaa,cod_prov,cod_mun,cod_sec,CCAA,Provincia,Municipio,Censo_Esc,Votos_Total,Participación,Nulos,Votos_Válidos,Blanco,V_Cand,PP,PSOE,Cs,UP,IU,VOX,UPyD,MP,CiU,ERC,JxC,CUP,DiL,PNV,Bildu,Amaiur,CC,FA,TE,BNG,PRC,GBai,Compromis,PACMA,Otros,...,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90-94,95-99,100 y más,Población Total,Hombres,Mujeres,% mayores 65 años,% 20-64 años,% menores 19 años,Afiliados SS Minicipio,% Afiliados SS autónomos,% Afiliados SS / Población,Paro Registrado Municipio,% Paro Hombres,% Paro mayores 45,% Paro s/ Afiliados SS Municipio,Renta persona 2017,Renta persona 2015,Renta hogar 2017,Renta hogar 2015,Renta Salarios 2018,Renta Salarios 2015,Renta Pensiones 2018,Renta Pensiones 2015,Renta Desempleo 2018,Renta Desempleo 2015,dict_res,dict_res_ord
0,022019111010400101001,01,04,04001,0400101001,Andalucía,Almería,Abla,1002,717,0.715569,7,710,3,707,193,310,47,30,0,122,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,2,...,73.0,80.0,89.0,81.0,94.0,87.0,91.0,77.0,72.0,42.0,67.0,56.0,19.0,4.0,0.0,1249.0,635.0,614.0,0.269816,0.590072,0.140112,304.0,0.223684,0.243395,140.0,0.421429,0.550000,0.315315,9159.0,8788.0,20172.0,19546.0,5574.0,4833.0,3286.0,3082.0,403.0,471.0,"{'PP': 193, 'PSOE': 310, 'Cs': 47, 'UP': 30, '...","[('PSOE', 310), ('PP', 193), ('VOX', 122), ('C..."
1,022019111010400201001,01,04,04002,0400201001,Andalucía,Almería,Abrucena,1013,711,0.701876,12,699,1,698,111,349,45,42,0,147,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,...,60.0,75.0,70.0,70.0,108.0,101.0,99.0,86.0,61.0,64.0,61.0,46.0,14.0,2.0,1.0,1202.0,637.0,565.0,0.278702,0.609817,0.111481,298.0,0.251678,0.247920,179.0,0.379888,0.625698,0.375262,8827.0,8107.0,17841.0,17115.0,4640.0,4048.0,3418.0,2770.0,568.0,620.0,"{'PP': 111, 'PSOE': 349, 'Cs': 45, 'UP': 42, '...","[('PSOE', 349), ('VOX', 147), ('PP', 111), ('C..."
2,022019111010400301001,01,04,04003,0400301001,Andalucía,Almería,Adra,667,484,0.725637,7,477,5,472,176,128,15,34,0,116,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,...,54.0,54.0,54.0,61.0,82.0,75.0,67.0,48.0,37.0,40.0,26.0,15.0,3.0,1.0,0.0,892.0,435.0,457.0,0.190583,0.643498,0.165919,7968.0,0.382530,8.932735,2525.0,0.432871,0.473663,0.240637,8965.0,8267.0,26498.0,24688.0,5121.0,4795.0,2499.0,2301.0,337.0,333.0,"{'PP': 176, 'PSOE': 128, 'Cs': 15, 'UP': 34, '...","[('PP', 176), ('PSOE', 128), ('VOX', 116), ('U..."
3,022019111010400301002,01,04,04003,0400301002,Andalucía,Almería,Adra,1306,909,0.696018,3,906,5,901,251,220,51,58,0,312,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,3,...,108.0,158.0,162.0,150.0,140.0,119.0,103.0,67.0,49.0,37.0,30.0,14.0,7.0,1.0,0.0,1752.0,865.0,887.0,0.117009,0.647260,0.235731,7968.0,0.382530,4.547945,2525.0,0.432871,0.473663,0.240637,8599.0,7941.0,25677.0,23400.0,5381.0,4837.0,1815.0,1724.0,343.0,464.0,"{'PP': 251, 'PSOE': 220, 'Cs': 51, 'UP': 58, '...","[('VOX', 312), ('PP', 251), ('PSOE', 220), ('U..."
4,022019111010400301003,01,04,04003,0400301003,Andalucía,Almería,Adra,1551,975,0.628627,12,963,9,954,292,202,73,52,0,327,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,3,...,189.0,178.0,215.0,227.0,164.0,110.0,96.0,61.0,58.0,41.0,40.0,27.0,4.0,4.0,0.0,2240.0,1094.0,1146.0,0.104911,0.647768,0.247321,7968.0,0.382530,3.557143,2525.0,0.432871,0.473663,0.240637,8076.0,7150.0,22051.0,19687.0,5224.0,4044.0,1170.0,1198.0,416.0,476.0,"{'PP': 292, 'PSOE': 202, 'Cs': 73, 'UP': 52, '...","[('VOX', 327), ('PP', 292), ('PSOE', 202), ('C..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36297,022019111195200108011,19,52,52001,5200108011,Melilla,Melilla,Melilla,1638,1021,0.623321,3,1018,11,1007,303,140,30,28,0,158,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,348,...,181.0,185.0,171.0,164.0,165.0,180.0,155.0,97.0,38.0,34.0,19.0,16.0,4.0,3.0,0.0,2480.0,1244.0,1236.0,0.0

A continuación sumamos los resultados de las secciones de España, normalizamos, y creamos la columna que será el vector 'y' en el modelo.

In [153]:
secciones_mod_lista = list(secciones_mod['Sección']) 

In [10]:
cols_validas_mod = ['Censo_Esc', 'Votos_Total', 'Nulos', 'Votos_Válidos', 'Blanco', 'V_Cand', 'PP', 'PSOE', 'Cs', 'UP',
       'IU', 'VOX', 'UPyD', 'MP', 'CiU', 'ERC', 'JxC', 'CUP', 'DiL', 'PNV',
       'Bildu', 'Amaiur', 'CC', 'FA', 'TE', 'BNG', 'PRC', 'GBai', 'Compromis',
       'PACMA', 'Otros']

In [154]:
secciones_mod = secciones_mod[cols_validas_mod]

In [155]:
secciones_mod

,Censo_Esc,Votos_Total,Nulos,Votos_Válidos,Blanco,V_Cand,PP,PSOE,Cs,UP,IU,VOX,UPyD,MP,CiU,ERC,JxC,CUP,DiL,PNV,Bildu,Amaiur,CC,FA,TE,BNG,PRC,GBai,Compromis,PACMA,Otros
0,1002,717,7,710,3,707,193,310,47,30,0,122,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,2
1,1013,711,12,699,1,698,111,349,45,42,0,147,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2
2,667,484,7,477,5,472,176,128,15,34,0,116,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0
3,1306,909,3,906,5,901,251,220,51,58,0,312,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,3
4,1551,975,12,963,9,954,292,202,73,52,0,327,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36297,1638,1021,3,1018,11,1007,303,140,30,28,0,158,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,348
36298,1676,1057,9,1048,2,1046,463,205,36,35,0,210,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,97
36299,1132,638,5,633,4,629,208,113,31,25,0,144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,108
36300,899,527,4,523,0,523,200,87,13,12,0,126,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,85


In [156]:
censo_mod = secciones_mod['Censo_Esc'].sum()

In [157]:
censo_mod

34871714

In [158]:
modelizacion = pd.DataFrame(secciones_mod.sum(), columns = ['Modelización'])

In [159]:
modelizacion['Modelización'] = modelizacion['Modelización'] / modelizacion['Modelización']['Censo_Esc']

Este será la columna y del modelo, una vez que quitemos la fila del censo, que siempre es 1 por construcción:

In [160]:
modelizacion

,Modelización
Censo_Esc,1.000000
Votos_Total,0.698612
Nulos,0.007127
Votos_Válidos,0.691485
Blanco,0.006201
V_Cand,0.685284
PP,0.146826
PSOE,0.193633
Cs,0.046953
UP,0.088816


In [161]:
modelizacion = modelizacion.drop(['Censo_Esc']) 

In [162]:
modelizacion

,Modelización
Votos_Total,0.698612
Nulos,0.007127
Votos_Válidos,0.691485
Blanco,0.006201
V_Cand,0.685284
PP,0.146826
PSOE,0.193633
Cs,0.046953
UP,0.088816
IU,0.000000


In [20]:
modelizacion.shape

(30, 1)

Ahora definimos las provinvias de donde seleccionaremos las secciones. Escogemos algunos muncipios pequeños de provincias o CCAA donde se presentaron partidos regionalistas o nacionalistas.

In [163]:
ccaa_select = []

provincia_select = []

municipio_select = ['Reus', 'Eibar', 'Laredo', 'Teruel', 'Paterna', 'Telde', 'Calatayud', 'Lucena', 'Verín']

secciones_select = df_eleccion_comp

In [164]:
if len(ccaa_select) > 0:

  secciones_select = secciones_select.loc[secciones_select['CCAA'].isin(ccaa_select)]

if len(provincia_select) > 0:

  secciones_select = secciones_select.loc[secciones_select['Provincia'].isin(provincia_select)]

if len(municipio_select) > 0:

  secciones_select = secciones_select.loc[secciones_select['Municipio'].isin(municipio_select)]



In [165]:
secciones_select

,Sección,cod_ccaa,cod_prov,cod_mun,cod_sec,CCAA,Provincia,Municipio,Censo_Esc,Votos_Total,Participación,Nulos,Votos_Válidos,Blanco,V_Cand,PP,PSOE,Cs,UP,IU,VOX,UPyD,MP,CiU,ERC,JxC,CUP,DiL,PNV,Bildu,Amaiur,CC,FA,TE,BNG,PRC,GBai,Compromis,PACMA,Otros,...,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90-94,95-99,100 y más,Población Total,Hombres,Mujeres,% mayores 65 años,% 20-64 años,% menores 19 años,Afiliados SS Minicipio,% Afiliados SS autónomos,% Afiliados SS / Población,Paro Registrado Municipio,% Paro Hombres,% Paro mayores 45,% Paro s/ Afiliados SS Municipio,Renta persona 2017,Renta persona 2015,Renta hogar 2017,Renta hogar 2015,Renta Salarios 2018,Renta Salarios 2015,Renta Pensiones 2018,Renta Pensiones 2015,Renta Desempleo 2018,Renta Desempleo 2015,dict_res,dict_res_ord
1791,022019111011403801001,01,14,14038,1403801001,Andalucía,Córdoba,Lucena,913,699,0.765608,9,690,3,687,263,97,65,53,0,205,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,...,52.0,63.0,77.0,78.0,90.0,75.0,65.0,67.0,68.0,56.0,39.0,32.0,18.0,2.0,0.0,1132.0,552.0,580.0,0.249117,0.559187,0.191696,17548.0,0.177114,15.501767,6685.0,0.334480,0.511294,0.275863,11025.0,10319.0,29436.0,28262.0,6487.0,5455.0,3588.0,3238.0,225.0,238.0,"{'PP': 263, 'PSOE': 97, 'Cs': 65, 'UP': 53, 'I...","[('PP', 263), ('VOX', 205), ('PSOE', 97), ('Cs..."
1792,022019111011403801002,01,14,14038,1403801002,Andalucía,Córdoba,Lucena,1023,725,0.708700,15,710,13,697,157,171,79,46,0,237,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,4,...,94.0,100.0,84.0,88.0,100.0,97.0,59.0,32.0,55.0,50.0,38.0,29.0,11.0,0.0,1.0,1291.0,643.0,648.0,0.167312,0.619675,0.213013,17548.0,0.177114,13.592564,6685.0,0.334480,0.511294,0.275863,8575.0,7770.0,22534.0,20780.0,5473.0,4532.0,2351.0,2333.0,353.0,482.0,"{'PP': 157, 'PSOE': 171, 'Cs': 79, 'UP': 46, '...","[('VOX', 237), ('PSOE', 171), ('PP', 157), ('C..."
1793,022019111011403801003,01,14,14038,1403801003,Andalucía,Córdoba,Lucena,1697,1218,0.717737,19,1199,14,1185,259,357,124,126,0,299,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,10,...,118.0,168.0,202.0,167.0,163.0,133.0,111.0,83.0,92.0,77.0,75.0,50.0,26.0,3.0,0.0,2213.0,1062.0,1151.0,0.183461,0.600994,0.215545,17548.0,0.177114,7.929507,6685.0,0.334480,0.511294,0.275863,7565.0,7095.0,19944.0,18935.0,5166.0,3985.0,2004.0,1940.0,378.0,483.0,"{'PP': 259, 'PSOE': 357, 'Cs': 124, 'UP': 126,...","[('PSOE', 357), ('VOX', 299), ('PP', 259), ('U..."
1794,022019111011403801004,01,14,14038,1403801004,Andalucía,Córdoba,Lucena,1872,1366,0.729701,18,1348,17,1331,233,339,158,124,0,456,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,13,...,212.0,245.0,243.0,196.0,188.0,140.0,123.0,68.0,54.0,54.0,37.0,15.0,8.0,0.0,0.0,2539.0,1268.0,1271.0,0.092950,0.657345,0.249705,17548.0,0.177114,6.911382,6685.0,0.334480,0.511294,0.275863,7386.0,6836.0,20709.0,19069.0,6228.0,4956.0,1207.0,1194.0,434.0,483.0,"{'PP': 233, 'PSOE': 339, 'Cs': 158, 'UP': 124,...","[('VOX', 456), ('PSOE', 339), ('PP', 233), ('C..."
1795,022019111011403801005,01,14,14038,1403801005,Andalucía,Córdoba,Lucena,844,647,0.766588,13,634,4,630,217,102,54,67,0,179,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,2,...,68.0,81.0,57.0,65.0,72.0,87.0,65.0,74.0,63.0,53.0,35.0,27.0,8.0,1.0,0.0,1081.0,537.0,544.0,0.241443,0.598520,0.160037,17548.0,0.177114,16.233117,6685.0,0.334480,0.511294,0.275863,10891.0,7902.0,28191.0,20660.0,5770.0,4814.0,3693.0,3470.0,258.0,295.0,"{'PP': 217, 'PSOE': 102, 'Cs': 54, 'UP': 67, '...","[('PP', 217), ('VOX', 179), ('PSOE', 102), ('U..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35281,022019111174619001044,17,46,46190,4619001044,La Rioja,Valencia,Paterna,1178,930,0.789474,6,924,12,912,172,202,140,101,0,219,0,64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,2,...,117.0,292.0,368.0,202.0,97.0,58.0,34.0,23.0,2

Comprobamos que tenemos las secciones de los nueve municipios.

In [166]:
secciones_select['Municipio'].unique()

array(['Lucena', 'Teruel', 'Calatayud', 'Telde', 'Laredo', 'Reus',
       'Verín', 'Eibar', 'Paterna'], dtype=object)

Nos quedamos con las secciones de más de 500 censados;  en este caso casi todas superan este límite.

In [167]:
secciones_select = secciones_select.loc[secciones_select['Censo_Esc'] > 500]

In [168]:
secciones_select

,Sección,cod_ccaa,cod_prov,cod_mun,cod_sec,CCAA,Provincia,Municipio,Censo_Esc,Votos_Total,Participación,Nulos,Votos_Válidos,Blanco,V_Cand,PP,PSOE,Cs,UP,IU,VOX,UPyD,MP,CiU,ERC,JxC,CUP,DiL,PNV,Bildu,Amaiur,CC,FA,TE,BNG,PRC,GBai,Compromis,PACMA,Otros,...,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90-94,95-99,100 y más,Población Total,Hombres,Mujeres,% mayores 65 años,% 20-64 años,% menores 19 años,Afiliados SS Minicipio,% Afiliados SS autónomos,% Afiliados SS / Población,Paro Registrado Municipio,% Paro Hombres,% Paro mayores 45,% Paro s/ Afiliados SS Municipio,Renta persona 2017,Renta persona 2015,Renta hogar 2017,Renta hogar 2015,Renta Salarios 2018,Renta Salarios 2015,Renta Pensiones 2018,Renta Pensiones 2015,Renta Desempleo 2018,Renta Desempleo 2015,dict_res,dict_res_ord
1791,022019111011403801001,01,14,14038,1403801001,Andalucía,Córdoba,Lucena,913,699,0.765608,9,690,3,687,263,97,65,53,0,205,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,...,52.0,63.0,77.0,78.0,90.0,75.0,65.0,67.0,68.0,56.0,39.0,32.0,18.0,2.0,0.0,1132.0,552.0,580.0,0.249117,0.559187,0.191696,17548.0,0.177114,15.501767,6685.0,0.334480,0.511294,0.275863,11025.0,10319.0,29436.0,28262.0,6487.0,5455.0,3588.0,3238.0,225.0,238.0,"{'PP': 263, 'PSOE': 97, 'Cs': 65, 'UP': 53, 'I...","[('PP', 263), ('VOX', 205), ('PSOE', 97), ('Cs..."
1792,022019111011403801002,01,14,14038,1403801002,Andalucía,Córdoba,Lucena,1023,725,0.708700,15,710,13,697,157,171,79,46,0,237,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,4,...,94.0,100.0,84.0,88.0,100.0,97.0,59.0,32.0,55.0,50.0,38.0,29.0,11.0,0.0,1.0,1291.0,643.0,648.0,0.167312,0.619675,0.213013,17548.0,0.177114,13.592564,6685.0,0.334480,0.511294,0.275863,8575.0,7770.0,22534.0,20780.0,5473.0,4532.0,2351.0,2333.0,353.0,482.0,"{'PP': 157, 'PSOE': 171, 'Cs': 79, 'UP': 46, '...","[('VOX', 237), ('PSOE', 171), ('PP', 157), ('C..."
1793,022019111011403801003,01,14,14038,1403801003,Andalucía,Córdoba,Lucena,1697,1218,0.717737,19,1199,14,1185,259,357,124,126,0,299,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,10,...,118.0,168.0,202.0,167.0,163.0,133.0,111.0,83.0,92.0,77.0,75.0,50.0,26.0,3.0,0.0,2213.0,1062.0,1151.0,0.183461,0.600994,0.215545,17548.0,0.177114,7.929507,6685.0,0.334480,0.511294,0.275863,7565.0,7095.0,19944.0,18935.0,5166.0,3985.0,2004.0,1940.0,378.0,483.0,"{'PP': 259, 'PSOE': 357, 'Cs': 124, 'UP': 126,...","[('PSOE', 357), ('VOX', 299), ('PP', 259), ('U..."
1794,022019111011403801004,01,14,14038,1403801004,Andalucía,Córdoba,Lucena,1872,1366,0.729701,18,1348,17,1331,233,339,158,124,0,456,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,13,...,212.0,245.0,243.0,196.0,188.0,140.0,123.0,68.0,54.0,54.0,37.0,15.0,8.0,0.0,0.0,2539.0,1268.0,1271.0,0.092950,0.657345,0.249705,17548.0,0.177114,6.911382,6685.0,0.334480,0.511294,0.275863,7386.0,6836.0,20709.0,19069.0,6228.0,4956.0,1207.0,1194.0,434.0,483.0,"{'PP': 233, 'PSOE': 339, 'Cs': 158, 'UP': 124,...","[('VOX', 456), ('PSOE', 339), ('PP', 233), ('C..."
1795,022019111011403801005,01,14,14038,1403801005,Andalucía,Córdoba,Lucena,844,647,0.766588,13,634,4,630,217,102,54,67,0,179,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,2,...,68.0,81.0,57.0,65.0,72.0,87.0,65.0,74.0,63.0,53.0,35.0,27.0,8.0,1.0,0.0,1081.0,537.0,544.0,0.241443,0.598520,0.160037,17548.0,0.177114,16.233117,6685.0,0.334480,0.511294,0.275863,10891.0,7902.0,28191.0,20660.0,5770.0,4814.0,3693.0,3470.0,258.0,295.0,"{'PP': 217, 'PSOE': 102, 'Cs': 54, 'UP': 67, '...","[('PP', 217), ('VOX', 179), ('PSOE', 102), ('U..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35281,022019111174619001044,17,46,46190,4619001044,La Rioja,Valencia,Paterna,1178,930,0.789474,6,924,12,912,172,202,140,101,0,219,0,64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,2,...,117.0,292.0,368.0,202.0,97.0,58.0,34.0,23.0,2

Llevamos a cabo el mismo proceso que en el cuaderno de la regresión lineal. Normalizamos inicialmente todas las secciones a la espera de encontrar las que serán válidas.

In [169]:
col_validas_select = ['Sección', 'Censo_Esc', 'Votos_Total', 'Nulos', 'Votos_Válidos', 'Blanco', 'V_Cand', 'PP', 'PSOE', 'Cs', 'UP',
       'IU', 'VOX', 'UPyD', 'MP', 'CiU', 'ERC', 'JxC', 'CUP', 'DiL', 'PNV',
       'Bildu', 'Amaiur', 'CC', 'FA', 'TE', 'BNG', 'PRC', 'GBai', 'Compromis',
       'PACMA', 'Otros']

In [170]:
secciones_select = secciones_select[col_validas_select]

In [171]:
secciones_select

,Sección,Censo_Esc,Votos_Total,Nulos,Votos_Válidos,Blanco,V_Cand,PP,PSOE,Cs,UP,IU,VOX,UPyD,MP,CiU,ERC,JxC,CUP,DiL,PNV,Bildu,Amaiur,CC,FA,TE,BNG,PRC,GBai,Compromis,PACMA,Otros
1791,022019111011403801001,913,699,9,690,3,687,263,97,65,53,0,205,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1
1792,022019111011403801002,1023,725,15,710,13,697,157,171,79,46,0,237,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,4
1793,022019111011403801003,1697,1218,19,1199,14,1185,259,357,124,126,0,299,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,10
1794,022019111011403801004,1872,1366,18,1348,17,1331,233,339,158,124,0,456,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,13
1795,022019111011403801005,844,647,13,634,4,630,217,102,54,67,0,179,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35281,022019111174619001044,1178,930,6,924,12,912,172,202,140,101,0,219,0,64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,2
35282,022019111174619001045,691,517,1,516,6,510,101,101,71,78,0,79,0,65,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,11
35283,022019111174619001046,1194,789,4,785,7,778,170,158,83,130,0,166,0,51,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,7
35284,022019111174619001047,1260,965,1,964,10,954,199,200,132,105,0,230,0,78,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,4


In [172]:
secciones_select_norm = secciones_select.copy()

In [173]:
secciones_select_norm

,Sección,Censo_Esc,Votos_Total,Nulos,Votos_Válidos,Blanco,V_Cand,PP,PSOE,Cs,UP,IU,VOX,UPyD,MP,CiU,ERC,JxC,CUP,DiL,PNV,Bildu,Amaiur,CC,FA,TE,BNG,PRC,GBai,Compromis,PACMA,Otros
1791,022019111011403801001,913,699,9,690,3,687,263,97,65,53,0,205,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1
1792,022019111011403801002,1023,725,15,710,13,697,157,171,79,46,0,237,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,4
1793,022019111011403801003,1697,1218,19,1199,14,1185,259,357,124,126,0,299,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,10
1794,022019111011403801004,1872,1366,18,1348,17,1331,233,339,158,124,0,456,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,13
1795,022019111011403801005,844,647,13,634,4,630,217,102,54,67,0,179,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35281,022019111174619001044,1178,930,6,924,12,912,172,202,140,101,0,219,0,64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,2
35282,022019111174619001045,691,517,1,516,6,510,101,101,71,78,0,79,0,65,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,11
35283,022019111174619001046,1194,789,4,785,7,778,170,158,83,130,0,166,0,51,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,7
35284,022019111174619001047,1260,965,1,964,10,954,199,200,132,105,0,230,0,78,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,4


In [174]:
  set_cols = ['Sección', 'Censo_Esc']

In [175]:
for col in secciones_select_norm.columns:

  if col not in set_cols:
    
    secciones_select_norm[col] = secciones_select_norm[col] / secciones_select_norm['Censo_Esc']

secciones_select_norm = secciones_select_norm.set_index('Sección')
secciones_select_norm = secciones_select_norm.drop('Censo_Esc', axis = 1)

secciones_select_norm = secciones_select_norm.T

In [176]:
secciones_select_norm

Sección,022019111011403801001,022019111011403801002,022019111011403801003,022019111011403801004,022019111011403801005,022019111011403801006,022019111011403802001,022019111011403802002,022019111011403802003,022019111011403802004,022019111011403802005,022019111011403802006,022019111011403802007,022019111011403802008,022019111011403802009,022019111011403802010,022019111011403803001,022019111011403803002,022019111011403803003,022019111011403803004,022019111011403803005,022019111011403803006,022019111011403803007,022019111011403803008,022019111011403804001,022019111011403805001,022019111011403805002,022019111024421601001,022019111024421601002,022019111024421602001,022019111024421602002,022019111024421602003,022019111024421602004,022019111024421602005,022019111024421602006,022019111024421602007,022019111024421603001,022019111024421603002,022019111024421603003,022019111024421603004,...,022019111174619001008,022019111174619001009,022019111174619001010,022019111174619001011,022019111174619001012,022019111174619001013,022019111174619001014,022019111174619001015,022019111174619001016,022019111174619001017,022019111174619001018,022019111174619001019,022019111174619001020,022019111174619001021,022019111174619001022,022019111174619001023,022019111174619001024,022019111174619001025,022019111174619001026,022019111174619001027,022019111174619001028,022019111174619001030,022019111174619001031,022019111174619001032,022019111174619001033,022019111174619001034,022019111174619001035,022019111174619001036,022019111174619001037,022019111174619001038,022019111174619001039,022019111174619001040,022019111174619001041,022019111174619001042,022019111174619001043,022019111174619001044,022019111174619001045,022019111174619001046,022019111174619001047,022019111174619001048
Votos_Total,0.765608,0.708700,0.717737,0.729701,0.766588,0.713405,0.697704,0.678420,0.700410,0.665417,0.746269,0.711618,0.659433,0.726496,0.721848,0.721251,0.714136,0.739054,0.725291,0.764157,0.711735,0.726077,0.753255,0.671593,0.630907,0.740924,0.690541,0.685422,0.700833,0.686798,0.765306,0.698686,0.740102,0.594123,0.755639,0.748603,0.737160,0.730196,0.760166,0.762027,...,0.711128,0.706651,0.733721,0.65375,0.725742,0.656692,0.635480,0.687005,0.640807,0.674275,0.630065,0.682953,0.660453,0.689715,0.679262,0.607973,0.682612,0.709276,0.549296,0.736752,0.784553,0.730212,0.803468,0.802704,0.338078,0.724638,0.785555,0.662346,0.805575,0.802002,0.304175,0.791696,0.774590,0.733570,0.784703,0.789474,0.748191,0.660804,0.765873,0.749107
Nulos,0.009858,0.014663,0.011196,0.009615,0.015403,0.018490,0.006378,0.012694,0.009362,0.023430,0.004975,0.011411,0.010850,0.013889,0.004812,0.017479,0.009459,0.016929,0.010174,0.003331,0.011480,0.010766,0.011719,0.008210,0.007024,0.024752,0.012162,0.004263,0.007500,0.007725,0.002268,0.003284,0.002030,0.000918,0.000627,0.002235,0.003021,0.001569,0.003320,0.002577,...,0.004573,0.007126,0.002326,0.00125,0.004812,0.006650,0.001215,0.003962,0.003879,0.009254,0.001307,0.004343,0.006658,0.006050,0.007030,0.001661,0.003037,0.002262,0.001408,0.001709,0.002033,0.003902,0.004817,0.006146,0.003559,0.004026,0.006701,0.005185,0.006272,0.007786,0.002982,0.002111,0.004098,0.006217,0.008499,0.005093,0.001447,0.003350,0.000794,0.001786
Votos_Válidos,0.755750,0.694037,0.706541,0.720085,0.751185,0.694915,0.691327,0.665726,0.691047,0.641987,0.741294,0.700207,0.648583,0.712607,0.717036,0.703772,0.704677,0.722125,0.715116,0.760826,0.700255,0.715311,0.741536,0.663383,0.623883,0.716172,0.678378,0.681159,0.693333,0.679073,0.763039,0.695402,0.738071,0.593205,0.755013,0.746369,0.734139,0.728627,0.756846,0.759450,...,0.706555,0.699525,0.731395,0.65250,0.720930,0.650042,0.634265,0.683043,0.636928,0.665022,0.628758,0.678610,0.653795,0.683665,0.672232,0.606312,0.679575,0.707014,0.547887,0.735043,0.782520,0.726310,0.798651,0.796558,0.334520,0.720612,0.778853,0.657161,0.799303,0.794216,0.301193,0.789585,0.770492,0.727353,0.776204,0.784380,0.746744,0.657454,0.765079,0.747321
Blanco,0.003286,0.012708,0.

Ahora definimos las dos funciones. La primera elimina los registros de los partidos que no se presentaron en esas provincias, y hace la trasposición y la normalización, y la segunda, que en base a la matriz de correlación va eliminando las que son demasiado similares entre sí.

In [177]:
def preparacion_sec(eleccion):

  set_cols = ['Sección', 'Censo_Esc']
  
  for col in eleccion.columns:

    if eleccion[col].sum() == 0:

      eleccion = eleccion.drop([col], axis = 1)

    elif col not in set_cols:

      eleccion[col] = eleccion[col] / eleccion['Censo_Esc']

  eleccion = eleccion.set_index('Sección')
  eleccion = eleccion.drop('Censo_Esc', axis = 1)

  df_elec_transpose = eleccion.T

  lista_sec = list(df_elec_transpose.columns)
  random.shuffle(lista_sec)

  df_elec_transpose = df_elec_transpose[lista_sec]

  return df_elec_transpose


In [178]:
def secciones_corr(dummy, threshold = 0.995):

  for ind in range(2, m.shape[0]):
    s = m.iloc[0:ind, 0:ind]

    if max(s.iloc[ind-1, 0:ind-1] > threshold):
    # print(m.columns[ind-1])
      dummy = dummy.drop(m.columns[ind-1], axis = 0)
      dummy = dummy.drop(m.columns[ind-1], axis = 1)

  return dummy.columns


In [179]:
secc = preparacion_sec(secciones_select)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [180]:
secc

Sección,022019111053502603009,022019111174619001001,022019111053502605003,022019111094312304005,022019111142003003001,022019111053502604002,022019111142003003004,022019111094312309002,022019111174619001044,022019111053502602003,022019111011403801003,022019111053502603008,022019111094312306008,022019111053502603011,022019111094312305007,022019111142003005004,022019111024421603006,022019111063903502001,022019111053502603010,022019111174619001012,022019111142003004002,022019111063903501002,022019111142003005003,022019111113208501001,022019111142003003003,022019111174619001007,022019111053502606015,022019111011403803002,022019111142003004006,022019111024421602005,022019111174619001025,022019111142003001005,022019111011403802006,022019111094312301002,022019111094312305003,022019111053502605007,022019111094312302009,022019111024421604001,022019111094312307005,022019111025006702005,...,022019111025006704001,022019111174619001034,022019111011403802004,022019111094312302010,022019111094312306013,022019111024421603010,022019111053502606013,022019111053502604003,022019111053502602008,022019111174619001045,022019111063903502002,022019111113208501010,022019111011403802009,022019111053502604001,022019111063903502004,022019111174619001028,022019111094312303011,022019111174619001031,022019111025006702001,022019111053502602005,022019111094312302013,022019111174619001021,022019111053502603016,022019111063903501001,022019111053502602004,022019111024421602007,022019111053502602010,022019111053502602007,022019111024421603011,022019111142003001004,022019111053502606008,022019111094312302006,022019111063903502003,022019111011403803003,022019111094312305005,022019111024421603009,022019111025006701005,022019111094312302007,022019111024421603005,022019111011403805001
Votos_Total,0.471947,0.713278,0.618966,0.726230,0.619231,0.592068,0.709779,0.519320,0.789474,0.598485,0.717737,0.583207,0.673404,0.494413,0.728893,0.677201,0.745137,0.744330,0.584983,0.725742,0.730220,0.693899,0.671902,0.609836,0.716198,0.761194,0.593060,0.739054,0.689295,0.594123,0.709276,0.684055,0.711618,0.724868,0.565008,0.670588,0.755862,0.724342,0.705580,0.755891,...,0.630673,0.724638,0.665417,0.718876,0.723982,0.827822,0.695431,0.592952,0.619940,0.748191,0.752395,0.652991,0.721848,0.599762,0.790776,0.784553,0.664612,0.803468,0.733143,0.605108,0.707595,0.689715,0.460425,0.753521,0.608240,0.748603,0.666667,0.560748,0.848875,0.653622,0.651070,0.704487,0.732489,0.725291,0.590504,0.812113,0.743415,0.731051,0.797030,0.740924
Nulos,0.006601,0.002350,0.006897,0.009836,0.003846,0.007082,0.001577,0.003091,0.005093,0.006629,0.011196,0.004539,0.003191,0.005587,0.004690,0.004515,0.006707,0.007216,0.005461,0.004812,0.003891,0.007239,0.003490,0.009836,0.001339,0.010333,0.007886,0.016929,0.001305,0.000918,0.002262,0.005906,0.011411,0.003023,0.000000,0.004278,0.008276,0.003947,0.003671,0.015080,...,0.006260,0.004026,0.023430,0.003347,0.003017,0.001140,0.004351,0.014097,0.003748,0.001447,0.005158,0.006838,0.004812,0.008314,0.008999,0.002033,0.003083,0.004817,0.014245,0.005894,0.003797,0.006050,0.003861,0.010060,0.007491,0.002235,0.013035,0.009346,0.001608,0.009159,0.010695,0.005128,0.006751,0.010174,0.003956,0.000688,0.011895,0.004890,0.002829,0.024752
Votos_Válidos,0.465347,0.710928,0.612069,0.716393,0.615385,0.584986,0.708202,0.516229,0.784380,0.591856,0.706541,0.578669,0.670213,0.488827,0.724203,0.672686,0.738431,0.737113,0.579522,0.720930,0.726329,0.686660,0.668412,0.600000,0.714859,0.750861,0.585174,0.722125,0.687990,0.593205,0.707014,0.678150,0.700207,0.721844,0.565008,0.666310,0.747586,0.720395,0.701909,0.740811,...,0.624413,0.720612,0.641987,0.715529,0.720965,0.826682,0.691080,0.578855,0.616192,0.746744,0.747237,0.646154,0.717036,0.591449,0.781777,0.782520,0.661529,0.798651,0.718898,0.599214,0.703797,0.683665,0.456564,0.743461,0.600749,0.746369,0.653631,0.551402,0.847267,0.644463,0.640374,0.699359,0.725738,0.715116,0.586548,0.811425,0.731521,0.726161,0.794201,0.716172
Blanco,0.005776,0.000000

Para ver cuán efectivo es el PCA hemos elegido un umbral poco exigente, por lo que obtenemos una lista de secciones distintas entre sí bastante amplio, unas 66.

In [182]:
m = secc.corr()
lista_sec = secciones_corr(m, 0.999)

In [183]:
lista_sec

Index(['022019111053502603009', '022019111174619001001',
       '022019111053502605003', '022019111094312304005',
       '022019111142003003001', '022019111053502604002',
       '022019111142003003004', '022019111174619001044',
       '022019111053502602003', '022019111011403801003',
       '022019111053502603008', '022019111094312306008',
       '022019111024421603006', '022019111063903502001',
       '022019111063903501002', '022019111142003005003',
       '022019111113208501001', '022019111142003003003',
       '022019111174619001007', '022019111011403803002',
       '022019111024421602005', '022019111142003001005',
       '022019111011403802006', '022019111094312301002',
       '022019111094312305003', '022019111094312302009',
       '022019111024421604001', '022019111025006702005',
       '022019111142003005002', '022019111174619001046',
       '022019111094312307001', '022019111011403801001',
       '022019111142003004004', '022019111053502606010',
       '022019111174619001019',

In [184]:
lista_sec.shape

(66,)

In [185]:
lista_sec = np.sort(lista_sec)

Ahora nos quedamos con las secciones antes mencionadas.

In [186]:
secciones_select_norm = secciones_select_norm[lista_sec]

In [187]:
secciones_select_norm

Sección,022019111011403801001,022019111011403801003,022019111011403802006,022019111011403803002,022019111011403803003,022019111011403803008,022019111011403804001,022019111011403805001,022019111011403805002,022019111024421602005,022019111024421603001,022019111024421603004,022019111024421603006,022019111024421603007,022019111024421603008,022019111024421603010,022019111024421604001,022019111025006702005,022019111053502602003,022019111053502603005,022019111053502603006,022019111053502603008,022019111053502603009,022019111053502604002,022019111053502605003,022019111053502605004,022019111053502606010,022019111063903501002,022019111063903502001,022019111063903502002,022019111063903502003,022019111063903503002,022019111094312301002,022019111094312302001,022019111094312302004,022019111094312302009,022019111094312302015,022019111094312303010,022019111094312304005,022019111094312305003,022019111094312306008,022019111094312307001,022019111094312308001,022019111094312308007,022019111113208501001,022019111142003001005,022019111142003002001,022019111142003002003,022019111142003003001,022019111142003003003,022019111142003003004,022019111142003004004,022019111142003005002,022019111142003005003,022019111174619001001,022019111174619001007,022019111174619001014,022019111174619001015,022019111174619001019,022019111174619001023,022019111174619001033,022019111174619001038,022019111174619001039,022019111174619001043,022019111174619001044,022019111174619001046
Votos_Total,0.765608,0.717737,0.711618,0.739054,0.725291,0.671593,0.630907,0.740924,0.690541,0.594123,0.737160,0.762027,0.745137,0.714032,0.711167,0.827822,0.724342,0.755891,0.598485,0.633880,0.563659,0.583207,0.471947,0.592068,0.618966,0.480534,0.595944,0.693899,0.744330,0.752395,0.732489,0.685652,0.724868,0.656514,0.626415,0.755862,0.817204,0.648130,0.726230,0.565008,0.673404,0.409565,0.620000,0.701431,0.609836,0.684055,0.659905,0.695983,0.619231,0.716198,0.709779,0.801541,0.715530,0.671902,0.713278,0.761194,0.635480,0.687005,0.682953,0.607973,0.338078,0.802002,0.304175,0.784703,0.789474,0.660804
Nulos,0.009858,0.011196,0.011411,0.016929,0.010174,0.008210,0.007024,0.024752,0.012162,0.000918,0.003021,0.002577,0.006707,0.003552,0.003119,0.001140,0.003947,0.015080,0.006629,0.009563,0.009889,0.004539,0.006601,0.007082,0.006897,0.011123,0.014561,0.007239,0.007216,0.005158,0.006751,0.003286,0.003023,0.001692,0.001887,0.008276,0.002688,0.005753,0.009836,0.000000,0.003191,0.002609,0.005333,0.005538,0.009836,0.005906,0.000000,0.004343,0.003846,0.001339,0.001577,0.003854,0.000000,0.003490,0.002350,0.010333,0.001215,0.003962,0.004343,0.001661,0.003559,0.007786,0.002982,0.008499,0.005093,0.003350
Votos_Válidos,0.755750,0.706541,0.700207,0.722125,0.715116,0.663383,0.623883,0.716172,0.678378,0.593205,0.734139,0.759450,0.738431,0.710480,0.708047,0.826682,0.720395,0.740811,0.591856,0.624317,0.553770,0.578669,0.465347,0.584986,0.612069,0.469410,0.581383,0.686660,0.737113,0.747237,0.725738,0.682366,0.721844,0.654822,0.624528,0.747586,0.814516,0.642378,0.716393,0.565008,0.670213,0.406957,0.614667,0.695893,0.600000,0.678150,0.659905,0.691640,0.615385,0.714859,0.708202,0.797688,0.715530,0.668412,0.710928,0.750861,0.634265,0.683043,0.678610,0.606312,0.334520,0.794216,0.301193,0.776204,0.784380,0.657454
Blanco,0.003286,0.008250,0.011411,0.006421,0.001453,0.004926,0.005747,0.004950,0.006757,0.003673,0.003021,0.000859,0.001341,0.000000,0.003119,0.004561,0.003947,0.004713,0.002841,0.005464,0.008653,0.003782,0.005776,0.003541,0.014368,0.000000,0.005200,0.001034,0.000000,0.000737,0.006751,0.003286,0.000756,0.006768,0.004717,0.001379,0.004032,0.002876,0.003279,0.006421,0.003191,0.000870,0.004000,0.005076,0.002459,0.002953,0.002387,0.007600,0.003077,0.008032,0.003155,0.007707,0.000000,0.002618,0.000000,0.005741,0.002430,0.009509,0.004343,0.006645,0.003559,0.003337,0.003976,0.011331,0.010187,0.005863
V_Cand,0.752464,0.698291,0.688797,0.715703,0.713663,0.658456,0.618135,0.711221,0.671622,0.589532,0.731118,0.758591,0.737089,0

In [188]:
secciones_select_norm.shape

(30, 66)

Ahora importamos la librería de PCA.

In [189]:
from sklearn.decomposition import PCA

Definimos la matriz X, y la transformamos en array numpy, como de costumbre.

In [190]:
X = secciones_select_norm.values

In [191]:
X

array([[0.76560789, 0.71773718, 0.71161826, ..., 0.78470255, 0.78947368,
        0.66080402],
       [0.00985761, 0.01119623, 0.01141079, ..., 0.00849858, 0.00509338,
        0.00335008],
       [0.75575027, 0.70654095, 0.70020747, ..., 0.77620397, 0.78438031,
        0.65745394],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00328587, 0.00589275, 0.00414938, ..., 0.01038716, 0.01018676,
        0.01088777],
       [0.00109529, 0.00589275, 0.00414938, ..., 0.00566572, 0.00169779,
        0.00586265]])

In [192]:
X.shape

(30, 66)

Instanciamos el modelo PCA con 4 componentes.

In [193]:
pca = PCA(n_components = 4)

Llevamos a cabo el fit y el transform.

In [194]:
pca.fit(X)

PCA(copy=True, iterated_power='auto', n_components=4, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [51]:
X_pca = pca.transform(X)

Como es de esperar, los primeros elementos del PCA tienen una proporción de la varianza mucho mayor que el resto.

In [195]:
print(pca.explained_variance_ratio_)

[0.96720314 0.01054905 0.00808517 0.00563125]


In [196]:
print(pca.singular_values_)

[8.820591   0.92118234 0.80646124 0.67304045]


Definimos el vector 'y'

In [197]:
y = modelizacion['Modelización'].values

In [198]:
y

array([6.98612262e-01, 7.12735256e-03, 6.91484910e-01, 6.20127247e-03,
       6.85283637e-01, 1.46825935e-01, 1.93632983e-01, 4.69532699e-02,
       8.88163685e-02, 0.00000000e+00, 1.04393406e-01, 0.00000000e+00,
       1.70579513e-02, 0.00000000e+00, 2.49424505e-02, 1.51066850e-02,
       7.02380158e-03, 0.00000000e+00, 1.08254501e-02, 7.93006618e-03,
       0.00000000e+00, 3.55525972e-03, 0.00000000e+00, 5.64956457e-04,
       3.42962781e-03, 1.96663691e-03, 3.61955251e-04, 0.00000000e+00,
       6.48608784e-03, 5.41074637e-03])

Ahora cargamos la librería de linear regression de Sklearn...

In [199]:
from sklearn.linear_model import LinearRegression

...y hacemos el fit, obteniendo un fit del 99,9% tomando únicamente los 4 primeros componentes.

In [200]:
reg = LinearRegression(fit_intercept = True).fit(X_pca, y)

In [201]:
reg.score(X_pca, y)

0.9995173580246297

Ahora podemos hacerlo un poco más sofisticado, utilizando un pipeline que incluya un scaler, un PCA con 10 componentes, y una regresión lineal. Además podemos ver el coeficiente R2.

In [202]:
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [203]:
from sklearn.metrics import mean_squared_error

In [204]:
from sklearn.metrics import r2_score

Definimos el pipeline

In [205]:
pipe_modelado = make_pipeline(StandardScaler(), PCA(n_components = 10), LinearRegression(fit_intercept=True))


Hacemos el fit

In [206]:
pipe_modelado.fit(X=X, y=y)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('pca',
                 PCA(copy=True, iterated_power='auto', n_components=10,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('linearregression',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)

Llevamos a cabo las predicciones con el metodo predict()

In [207]:
predicciones = pipe_modelado.predict(X=X)
predicciones = predicciones.flatten()

In [208]:
r2 = r2_score(
            y_true  = y,
            y_pred  = predicciones
           )

El coeficiente R2 resulta ser altísimo, del 99.996%.

In [209]:
r2

0.9999623518769771

Ahora podemos comparar las predicciones con los datos reales. Primero deshacemos la normalización multiplicando por el censo de España.

In [210]:
est = predicciones * censo_mod

In [211]:
df = pd.DataFrame(est, index = secciones_select_norm.index, columns = ['Estimación']).astype('int32')

In [212]:
df

,Estimación
Votos_Total,24359895
Nulos,253469
Votos_Válidos,24105725
Blanco,178452
V_Cand,23926573
PP,5072187
PSOE,6749963
Cs,1796615
UP,2986509
IU,-699


In [213]:
df1 = pd.DataFrame(secciones_mod.sum(), columns = ['Real']).drop('Censo_Esc')

La comparación con los datos reales es bastante satisfactoria, pues las diferencias son muy pequeñas respecto a los datos reales. Hay que tener en cuenta que hemos seleccionado 66 secciones, tomadas a su vez de municipios escogidos arbitrariamente. 

In [214]:
df['Real'] = df1['Real']

In [216]:
df['pc Estimación'] = df['Estimación'] / df['Estimación'][2] * 100

In [217]:
df['pc Real'] = df['Real'] / df['Real'][2] * 100

In [218]:
df['dif. Real-Est.'] = df['pc Real'] - df['pc Estimación']

In [219]:
df

,Estimación,Real,pc Estimación,pc Real,dif. Real-Est.
Votos_Total,24359895,24361807,101.054397,101.030731,-0.023665
Nulos,253469,248543,1.051489,1.030731,-0.020757
Votos_Válidos,24105725,24113264,100.000000,100.000000,0.000000
Blanco,178452,216249,0.740289,0.896805,0.156516
V_Cand,23926573,23897015,99.256807,99.103195,-0.153612
PP,5072187,5120072,21.041421,21.233426,0.192005
PSOE,6749963,6752314,28.001493,28.002489,0.000995
Cs,1796615,1637341,7.453064,6.790209,-0.662855
UP,2986509,3097179,12.389210,12.844296,0.455086
IU,-699,0,-0.002900,0.000000,0.002900


## Modelización en las elecciones de 2016

Ahora vamos a ver lo bien o mal que se ajusta el modelo del pipeline si lo aplicamos a las elecciones de junio de 2016, considerando las secciones equivalente a las 66 utilizadas para estimar el resultado de noviembre de 2019. 

Comenzamos cargando el dataset de equivalencia de las secciones.

In [220]:
sim_secciones = pd.read_csv('/content/drive/MyDrive/Proyecto_KeepCoding - Propio/Data/similitud_secciones_def_REF.csv', dtype = 'str')

Ahora seleccinamos las similares a las 66 secciones que encontramos en el capítulo anterior...

In [221]:
sec_select_J16 = sim_secciones.loc[sim_secciones['cod_sec_ref'].isin(lista_sec)]

In [222]:
sec_select_J16.shape

(66, 12)

... y escogemos sus equivalentes en las elecciones de 2016, que son estas 66:

In [223]:
list_sec_J16 = list(sec_select_J16['cercana J16_ref'])

In [224]:
list_sec_J16 = np.sort(list_sec_J16)

In [225]:
list_sec_J16.shape

(66,)

In [226]:
secciones_select_norm = secciones_select_norm[lista_sec]

In [227]:
secciones_select_norm

Sección,022019111011403801001,022019111011403801003,022019111011403802006,022019111011403803002,022019111011403803003,022019111011403803008,022019111011403804001,022019111011403805001,022019111011403805002,022019111024421602005,022019111024421603001,022019111024421603004,022019111024421603006,022019111024421603007,022019111024421603008,022019111024421603010,022019111024421604001,022019111025006702005,022019111053502602003,022019111053502603005,022019111053502603006,022019111053502603008,022019111053502603009,022019111053502604002,022019111053502605003,022019111053502605004,022019111053502606010,022019111063903501002,022019111063903502001,022019111063903502002,022019111063903502003,022019111063903503002,022019111094312301002,022019111094312302001,022019111094312302004,022019111094312302009,022019111094312302015,022019111094312303010,022019111094312304005,022019111094312305003,022019111094312306008,022019111094312307001,022019111094312308001,022019111094312308007,022019111113208501001,022019111142003001005,022019111142003002001,022019111142003002003,022019111142003003001,022019111142003003003,022019111142003003004,022019111142003004004,022019111142003005002,022019111142003005003,022019111174619001001,022019111174619001007,022019111174619001014,022019111174619001015,022019111174619001019,022019111174619001023,022019111174619001033,022019111174619001038,022019111174619001039,022019111174619001043,022019111174619001044,022019111174619001046
Votos_Total,0.765608,0.717737,0.711618,0.739054,0.725291,0.671593,0.630907,0.740924,0.690541,0.594123,0.737160,0.762027,0.745137,0.714032,0.711167,0.827822,0.724342,0.755891,0.598485,0.633880,0.563659,0.583207,0.471947,0.592068,0.618966,0.480534,0.595944,0.693899,0.744330,0.752395,0.732489,0.685652,0.724868,0.656514,0.626415,0.755862,0.817204,0.648130,0.726230,0.565008,0.673404,0.409565,0.620000,0.701431,0.609836,0.684055,0.659905,0.695983,0.619231,0.716198,0.709779,0.801541,0.715530,0.671902,0.713278,0.761194,0.635480,0.687005,0.682953,0.607973,0.338078,0.802002,0.304175,0.784703,0.789474,0.660804
Nulos,0.009858,0.011196,0.011411,0.016929,0.010174,0.008210,0.007024,0.024752,0.012162,0.000918,0.003021,0.002577,0.006707,0.003552,0.003119,0.001140,0.003947,0.015080,0.006629,0.009563,0.009889,0.004539,0.006601,0.007082,0.006897,0.011123,0.014561,0.007239,0.007216,0.005158,0.006751,0.003286,0.003023,0.001692,0.001887,0.008276,0.002688,0.005753,0.009836,0.000000,0.003191,0.002609,0.005333,0.005538,0.009836,0.005906,0.000000,0.004343,0.003846,0.001339,0.001577,0.003854,0.000000,0.003490,0.002350,0.010333,0.001215,0.003962,0.004343,0.001661,0.003559,0.007786,0.002982,0.008499,0.005093,0.003350
Votos_Válidos,0.755750,0.706541,0.700207,0.722125,0.715116,0.663383,0.623883,0.716172,0.678378,0.593205,0.734139,0.759450,0.738431,0.710480,0.708047,0.826682,0.720395,0.740811,0.591856,0.624317,0.553770,0.578669,0.465347,0.584986,0.612069,0.469410,0.581383,0.686660,0.737113,0.747237,0.725738,0.682366,0.721844,0.654822,0.624528,0.747586,0.814516,0.642378,0.716393,0.565008,0.670213,0.406957,0.614667,0.695893,0.600000,0.678150,0.659905,0.691640,0.615385,0.714859,0.708202,0.797688,0.715530,0.668412,0.710928,0.750861,0.634265,0.683043,0.678610,0.606312,0.334520,0.794216,0.301193,0.776204,0.784380,0.657454
Blanco,0.003286,0.008250,0.011411,0.006421,0.001453,0.004926,0.005747,0.004950,0.006757,0.003673,0.003021,0.000859,0.001341,0.000000,0.003119,0.004561,0.003947,0.004713,0.002841,0.005464,0.008653,0.003782,0.005776,0.003541,0.014368,0.000000,0.005200,0.001034,0.000000,0.000737,0.006751,0.003286,0.000756,0.006768,0.004717,0.001379,0.004032,0.002876,0.003279,0.006421,0.003191,0.000870,0.004000,0.005076,0.002459,0.002953,0.002387,0.007600,0.003077,0.008032,0.003155,0.007707,0.000000,0.002618,0.000000,0.005741,0.002430,0.009509,0.004343,0.006645,0.003559,0.003337,0.003976,0.011331,0.010187,0.005863
V_Cand,0.752464,0.698291,0.688797,0.715703,0.713663,0.658456,0.618135,0.711221,0.671622,0.589532,0.731118,0.758591,0.737089,0

Cargamos ahora los resultados de las elecciones de junio de 2016

In [228]:
df_eleccion_comp_J16 = pd.read_csv('/content/drive/MyDrive/Proyecto_KeepCoding - Propio/Data/Gen-16-Jun/gen_J16_unif_cols_prov.txt', dtype = strings)

Seleccionamos las secciones a modelizar, que son naturalmente las de toda España.

In [229]:
secciones_mod = df_eleccion_comp_J16

if len(ccaa_mod) > 0:

  secciones_mod = secciones_mod.loc[secciones_mod['CCAA'].isin(ccaa_mod)]

if len(provincia_mod) > 0:

  secciones_mod = secciones_mod.loc[secciones_mod['Provincia'].isin(provincia_mod)]

if len(municipio_mod) > 0:

  secciones_mod = secciones_mod.loc[secciones_mod['Municipio'].isin(municipio_mod)]

In [230]:
secciones_mod

,Sección,cod_ccaa,cod_prov,cod_mun,cod_sec,CCAA,Provincia,Municipio,Censo_Esc,Votos_Total,Participación,Nulos,Votos_Válidos,Blanco,V_Cand,PP,PSOE,Cs,UP,IU,VOX,UPyD,MP,CiU,ERC,JxC,CUP,DiL,PNV,Bildu,Amaiur,CC,FA,TE,BNG,PRC,GBai,Compromis,PACMA,Otros,...,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90-94,95-99,100 y más,Población Total,Hombres,Mujeres,% mayores 65 años,% 20-64 años,% menores 19 años,Afiliados SS Minicipio,% Afiliados SS autónomos,% Afiliados SS / Población,Paro Registrado Municipio,% Paro Hombres,% Paro mayores 45,% Paro s/ Afiliados SS Municipio,Renta persona 2017,Renta persona 2015,Renta hogar 2017,Renta hogar 2015,Renta Salarios 2018,Renta Salarios 2015,Renta Pensiones 2018,Renta Pensiones 2015,Renta Desempleo 2018,Renta Desempleo 2015,dict_res,dict_res_ord
0,022016061010400101001,01,04,04001,0400101001,Andalucía,Almería,Abla,1062,823,0.774953,9,814,5,809,267,356,110,65,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,3,...,66.0,92.0,87.0,83.0,104.0,88.0,91.0,64.0,61.0,72.0,89.0,42.0,13.0,0.0,1.0,1294.0,664.0,630.0,0.264297,0.594281,0.141422,267.0,0.292135,0.206337,184.0,0.461957,0.548913,0.407982,9159.0,8788.0,20172.0,19546.0,5574.0,4833.0,3286.0,3082.0,403.0,471.0,"{'PP': 267, 'PSOE': 356, 'Cs': 110, 'UP': 65, ...","[('PSOE', 356), ('PP', 267), ('Cs', 110), ('UP..."
1,022016061010400201001,01,04,04002,0400201001,Andalucía,Almería,Abrucena,1040,748,0.719231,8,740,2,738,212,342,93,79,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,6,...,63.0,73.0,65.0,91.0,97.0,95.0,106.0,61.0,57.0,74.0,67.0,42.0,8.0,2.0,0.0,1208.0,615.0,593.0,0.257450,0.612583,0.129967,331.0,0.238671,0.274007,183.0,0.437158,0.573770,0.356031,8827.0,8107.0,17841.0,17115.0,4640.0,4048.0,3418.0,2770.0,568.0,620.0,"{'PP': 212, 'PSOE': 342, 'Cs': 93, 'UP': 79, '...","[('PSOE', 342), ('PP', 212), ('Cs', 93), ('UP'..."
2,022016061010400301001,01,04,04003,0400301001,Andalucía,Almería,Adra,666,487,0.731231,7,480,2,478,266,112,48,46,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,...,54.0,56.0,53.0,76.0,82.0,72.0,52.0,42.0,36.0,35.0,28.0,4.0,3.0,0.0,1.0,878.0,421.0,457.0,0.169704,0.636674,0.193622,7032.0,0.404863,8.009112,3663.0,0.395304,0.428610,0.342496,8965.0,8267.0,26498.0,24688.0,5121.0,4795.0,2499.0,2301.0,337.0,333.0,"{'PP': 266, 'PSOE': 112, 'Cs': 48, 'UP': 46, '...","[('PP', 266), ('PSOE', 112), ('Cs', 48), ('UP'..."
3,022016061010400301002,01,04,04003,0400301002,Andalucía,Almería,Adra,1264,867,0.685918,3,864,4,860,436,211,102,101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,...,146.0,179.0,139.0,130.0,133.0,122.0,84.0,44.0,44.0,39.0,18.0,15.0,3.0,0.0,0.0,1693.0,839.0,854.0,0.096279,0.669226,0.234495,7032.0,0.404863,4.153574,3663.0,0.395304,0.428610,0.342496,8599.0,7941.0,25677.0,23400.0,5381.0,4837.0,1815.0,1724.0,343.0,464.0,"{'PP': 436, 'PSOE': 211, 'Cs': 102, 'UP': 101,...","[('PP', 436), ('PSOE', 211), ('Cs', 102), ('UP..."
4,022016061010400301003,01,04,04003,0400301003,Andalucía,Almería,Adra,1439,952,0.661571,9,943,10,933,512,214,111,85,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,3,...,173.0,209.0,234.0,166.0,142.0,91.0,74.0,60.0,49.0,46.0,48.0,13.0,8.0,1.0,0.0,2149.0,1037.0,1112.0,0.104700,0.645882,0.249418,7032.0,0.404863,3.272220,3663.0,0.395304,0.428610,0.342496,8076.0,7150.0,22051.0,19687.0,5224.0,4044.0,1170.0,1198.0,416.0,476.0,"{'PP': 512, 'PSOE': 214, 'Cs': 111, 'UP': 85, ...","[('PP', 512), ('PSOE', 214), ('Cs', 111), ('UP..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36188,022016061195200108011,19,52,52001,5200108011,Melilla,Melilla,Melilla,1510,860,0.569536,12,848,6,842,401,172,158,98,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,...,173.0,165.0,158.0,162.0,150.0,181.0,120.0,68.0,40.0,35.0,21.0,15.0,6.0,1.0,0.0,2296.0,1141.0,1155.0,0.081010,0.633275,0

In [231]:
censo_mod = secciones_mod['Censo_Esc'].sum()

In [232]:
censo_mod

34595051

Procedemos de igual manera, sumamos los resultados, normalizamos y los almacenamos en un df.

In [233]:
secciones_mod = secciones_mod[cols_validas_mod]

In [234]:
modelizacion = pd.DataFrame(secciones_mod.sum(), columns = ['Modelización'])
modelizacion['Modelización'] = modelizacion['Modelización'] / modelizacion['Modelización']['Censo_Esc']
modelizacion = modelizacion.drop(['Censo_Esc']) 

In [235]:
modelizacion

,Modelización
Votos_Total,0.698307
Nulos,0.006491
Votos_Válidos,0.691816
Blanco,0.005161
V_Cand,0.686655
PP,0.228552
PSOE,0.156789
Cs,0.090286
UP,0.146014
IU,0.000000


In [236]:
modelizacion.shape

(30, 1)

Comprobamos que el número de secciones equivalentes es el mismo, 66, que está esperando el modelo. Esto es importante porque en uno de las pruebas no fue así. La razón fue que dos de las secciones de 2019 tenían como equivalente en 2016 la misma sección. En ese caso no pudimos ejecutar el método .isin(), sino que echamos mano de un for loop.

In [237]:
np.unique(list_sec_J16).shape

(66,)

In [238]:
secciones_select = df_eleccion_comp_J16.loc[df_eleccion_comp_J16['Sección'].isin(list_sec_J16)]

In [239]:
secciones_select = secciones_select[col_validas_select]

In [240]:
secciones_select

,Sección,Censo_Esc,Votos_Total,Nulos,Votos_Válidos,Blanco,V_Cand,PP,PSOE,Cs,UP,IU,VOX,UPyD,MP,CiU,ERC,JxC,CUP,DiL,PNV,Bildu,Amaiur,CC,FA,TE,BNG,PRC,GBai,Compromis,PACMA,Otros
1765,022016061011403801001,884,689,7,682,2,680,426,79,104,67,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
1767,022016061011403801003,1743,1180,9,1171,15,1156,450,377,170,144,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,5
1776,022016061011403802006,945,637,9,628,8,620,273,181,102,52,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,3
1782,022016061011403803002,1682,1177,15,1162,9,1153,410,410,215,95,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,6
1783,022016061011403803003,710,472,4,468,2,466,179,194,54,33,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35164,022016061174619001038,856,682,3,679,3,676,254,105,139,152,0,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,9
35165,022016061174619001039,983,365,5,360,3,357,63,115,26,139,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,5
35169,022016061174619001043,997,786,3,783,3,780,286,105,226,141,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,13
35170,022016061174619001044,1099,883,4,879,3,876,279,122,226,217,0,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17,10


In [241]:
secciones_select.shape

(66, 32)

Hacemos un pequeño inciso para explicar que hicimos en el caso de no poder utilizar .isin().

In [110]:
secciones_select.iloc[0, ]

Sección          022016061011403801001
Censo_Esc                          884
Votos_Total                        689
Nulos                                7
Votos_Válidos                      682
Blanco                               2
V_Cand                             680
PP                                 426
PSOE                                79
Cs                                 104
UP                                  67
IU                                   0
VOX                                  1
UPyD                                 1
MP                                   0
CiU                                  0
ERC                                  0
JxC                                  0
CUP                                  0
DiL                                  0
PNV                                  0
Bildu                                0
Amaiur                               0
CC                                   0
FA                                   0
TE                       

En ese caso creamos un dataframe dummy con un elemento inicial que defina así las columnas, dff.



In [118]:
dff = pd.DataFrame(secciones_select.iloc[0, ]).T

In [119]:
dff

,Sección,Censo_Esc,Votos_Total,Nulos,Votos_Válidos,Blanco,V_Cand,PP,PSOE,Cs,UP,IU,VOX,UPyD,MP,CiU,ERC,JxC,CUP,DiL,PNV,Bildu,Amaiur,CC,FA,TE,BNG,PRC,GBai,Compromis,PACMA,Otros
1765,022016061011403801001,884,689,7,682,2,680,426,79,104,67,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1


... y aplicamos un for loop para seleccionar las filas de las secciones equivalentes

In [120]:
for sec in list_sec_J16:

  row = df_eleccion_comp_J16.loc[df_eleccion_comp_J16['Sección'] == sec]

  dff = dff.append(row)



In [121]:
dff

,Sección,Censo_Esc,Votos_Total,Nulos,Votos_Válidos,Blanco,V_Cand,PP,PSOE,Cs,UP,IU,VOX,UPyD,MP,CiU,ERC,JxC,CUP,DiL,PNV,Bildu,Amaiur,CC,FA,TE,BNG,PRC,GBai,Compromis,PACMA,Otros,cod_ccaa,cod_prov,cod_mun,cod_sec,CCAA,Provincia,Municipio,Participación,...,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85-89,90-94,95-99,100 y más,Población Total,Hombres,Mujeres,% mayores 65 años,% 20-64 años,% menores 19 años,Afiliados SS Minicipio,% Afiliados SS autónomos,% Afiliados SS / Población,Paro Registrado Municipio,% Paro Hombres,% Paro mayores 45,% Paro s/ Afiliados SS Municipio,Renta persona 2017,Renta persona 2015,Renta hogar 2017,Renta hogar 2015,Renta Salarios 2018,Renta Salarios 2015,Renta Pensiones 2018,Renta Pensiones 2015,Renta Desempleo 2018,Renta Desempleo 2015,dict_res,dict_res_ord
1765,022016061011403801001,884,689,7,682,2,680,426,79,104,67,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1765,022016061011403801001,884,689,7,682,2,680,426,79,104,67,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,01,14,14038,1403801001,Andalucía,Córdoba,Lucena,0.779412,...,49.0,72.0,81.0,78.0,86.0,70.0,63.0,67.0,57.0,42.0,45.0,35.0,14.0,5.0,0.0,1124.0,541.0,583.0,0.235765,0.577402,0.186833,14787.0,0.204031,13.155694,7858.0,0.377704,0.461441,0.347008,11025.0,10319.0,29436.0,28262.0,6487.0,5455.0,3588.0,3238.0,225.0,238.0,"{'PP': 426, 'PSOE': 79, 'Cs': 104, 'UP': 67, '...","[('PP', 426), ('Cs', 104), ('PSOE', 79), ('UP'..."
1767,022016061011403801003,1743,1180,9,1171,15,1156,450,377,170,144,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,5,01,14,14038,1403801003,Andalucía,Córdoba,Lucena,0.676994,...,144.0,206.0,174.0,172.0,152.0,123.0,93.0,89.0,79.0,84.0,80.0,46.0,28.0,1.0,0.0,2270.0,1093.0,1177.0,0.179295,0.596035,0.224670,14787.0,0.204031,6.514097,7858.0,0.377704,0.461441,0.347008,7565.0,7095.0,19944.0,18935.0,5166.0,3985.0,2004.0,1940.0,378.0,483.0,"{'PP': 450, 'PSOE': 377, 'Cs': 170, 'UP': 144,...","[('PP', 450), ('PSOE', 377), ('Cs', 170), ('UP..."
1770,022016061011403801006,673,448,0,448,1,447,190,121,77,51,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,4,01,14,14038,1403801006,Andalucía,Córdoba,Lucena,0.665676,...,59.0,79.0,72.0,58.0,58.0,39.0,38.0,33.0,42.0,38.0,33.0,21.0,10.0,4.0,0.0,875.0,432.0,443.0,0.206857,0.581714,0.211429,14787.0,0.204031,16.899429,7858.0,0.377704,0.461441,0.347008,7645.0,7696.0,19281.0,19901.0,5199.0,4306.0,2269.0,2247.0,371.0,392.0,"{'PP': 190, 'PSOE': 121, 'Cs': 77, 'UP': 51, '...","[('PP', 190), ('PSOE', 121), ('Cs', 77), ('UP'..."
1774,022016061011403802004,1050,678,12,666,5,661,200,252,103,96,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,2,01,14,14038,1403802004,Andalucía,Córdoba,Lucena,0.645714,...,71.0,129.0,164.0,165.0,115.0,78.0,41.0,36.0,48.0,30.0,23.0,14.0,2.0,1.0,0.0,1488.0,758.0,730.0,0.103495,0.620968,0.275538,14787.0,0.204031,9.937500,7858.0,0.377704,0.461441,0.347008,6266.0,5786.0,17705.0,16493.0,4644.0,3522.0,1420.0,1343.0,494.0,595.0,"{'PP': 200, 'PSOE': 252, 'Cs': 103, 'UP': 96, ...","[('PSOE', 252), ('PP', 200), ('Cs', 103), ('UP..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35161,022016061174619001035,1347,1101,0,1101,6,1095,501,133,237,196,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,11,17,46,46190,4619001035,La Rioja,Valencia,Paterna,0.817372,...,84.0,139.0,131.0,164.0,155.0,161.0,135.0,87.0,53.0,28.0,15.0,18.0,7.0,4.0,1.0,1824.0,945.0,879.0,0.116776,0.632675,0.250548,39890.0,0.111782,21.869518,8268.0,0.428882,0.450895,0.171685,18111.0,17313.0,54880.0,53765.0,15839.0,14414.0,3167.0,2473.0,198.0,256.0,"{'PP': 501, 'PSOE': 133, 'Cs': 237,

... Y nos quedamos con todas excepto la primera que usamos para crear el dummy.

In [122]:
secciones_select = dff.iloc[1:,]

Pero en este caso no es necesario.

In [242]:
secciones_select

,Sección,Censo_Esc,Votos_Total,Nulos,Votos_Válidos,Blanco,V_Cand,PP,PSOE,Cs,UP,IU,VOX,UPyD,MP,CiU,ERC,JxC,CUP,DiL,PNV,Bildu,Amaiur,CC,FA,TE,BNG,PRC,GBai,Compromis,PACMA,Otros
1765,022016061011403801001,884,689,7,682,2,680,426,79,104,67,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
1767,022016061011403801003,1743,1180,9,1171,15,1156,450,377,170,144,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,5
1776,022016061011403802006,945,637,9,628,8,620,273,181,102,52,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,3
1782,022016061011403803002,1682,1177,15,1162,9,1153,410,410,215,95,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,6
1783,022016061011403803003,710,472,4,468,2,466,179,194,54,33,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35164,022016061174619001038,856,682,3,679,3,676,254,105,139,152,0,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,9
35165,022016061174619001039,983,365,5,360,3,357,63,115,26,139,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,5
35169,022016061174619001043,997,786,3,783,3,780,286,105,226,141,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,13
35170,022016061174619001044,1099,883,4,879,3,876,279,122,226,217,0,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17,10


In [243]:
secciones_select_norm = secciones_select.copy()

Y ahora simplemente normalizamos y trasponemos.

In [244]:
for col in secciones_select_norm.columns:

  if col not in set_cols:
    
    secciones_select_norm[col] = secciones_select_norm[col] / secciones_select_norm['Censo_Esc']

secciones_select_norm = secciones_select_norm.set_index('Sección')
secciones_select_norm = secciones_select_norm.drop('Censo_Esc', axis = 1)

secciones_select_norm = secciones_select_norm.T

In [245]:
secciones_select_norm

Sección,022016061011403801001,022016061011403801003,022016061011403802006,022016061011403803002,022016061011403803003,022016061011403803008,022016061011403804001,022016061011403805001,022016061011403805002,022016061024421602005,022016061024421603001,022016061024421603004,022016061024421603006,022016061024421603007,022016061024421603008,022016061024421603010,022016061024421604001,022016061025006702005,022016061053502602003,022016061053502603005,022016061053502603006,022016061053502603008,022016061053502603009,022016061053502604002,022016061053502605003,022016061053502605004,022016061053502606010,022016061063903501002,022016061063903502001,022016061063903502003,022016061063903502005,022016061063903503002,022016061094312301002,022016061094312302001,022016061094312302004,022016061094312302009,022016061094312302015,022016061094312303010,022016061094312304005,022016061094312305003,022016061094312306008,022016061094312307001,022016061094312308001,022016061094312308007,022016061113208501001,022016061142003001005,022016061142003002001,022016061142003002003,022016061142003003001,022016061142003003003,022016061142003003004,022016061142003004004,022016061142003005002,022016061142003005003,022016061174619001001,022016061174619001007,022016061174619001014,022016061174619001015,022016061174619001019,022016061174619001023,022016061174619001033,022016061174619001038,022016061174619001039,022016061174619001043,022016061174619001044,022016061174619001046
Votos_Total,0.779412,0.676994,0.674074,0.699762,0.664789,0.651399,0.629556,0.746622,0.717105,0.613321,0.679719,0.762215,0.739779,0.661352,0.696521,0.783590,0.695652,0.752363,0.640118,0.694051,0.635152,0.630137,0.554688,0.638589,0.672225,0.539754,0.633117,0.689349,0.780039,0.759434,0.625650,0.706422,0.680219,0.590494,0.553571,0.704420,0.726902,0.573709,0.675941,0.559801,0.584416,0.401802,0.580282,0.637044,0.686579,0.651558,0.636577,0.688525,0.581624,0.678981,0.658647,0.742593,0.679767,0.673276,0.683041,0.780488,0.625882,0.724026,0.702820,0.635762,0.391254,0.796729,0.371312,0.788365,0.803458,0.689805
Nulos,0.007919,0.005164,0.009524,0.008918,0.005634,0.003393,0.004639,0.006757,0.013158,0.013876,0.007028,0.006515,0.004543,0.013194,0.010363,0.008260,0.012422,0.009452,0.007866,0.012748,0.010909,0.008371,0.007031,0.011519,0.010926,0.005599,0.005952,0.003945,0.017442,0.011792,0.005199,0.008519,0.007819,0.000000,0.004699,0.005525,0.001359,0.005634,0.009820,0.008306,0.004329,0.000901,0.000000,0.002996,0.015548,0.001889,0.001172,0.003074,0.004102,0.001274,0.004511,0.001852,0.005822,0.003448,0.002339,0.008130,0.007059,0.004870,0.017354,0.003311,0.002301,0.003505,0.005086,0.003009,0.003640,0.002169
Votos_Válidos,0.771493,0.671830,0.664550,0.690844,0.659155,0.648007,0.624917,0.739865,0.703947,0.599445,0.672691,0.755700,0.735237,0.648158,0.686158,0.775330,0.683230,0.742911,0.632252,0.681303,0.624242,0.621766,0.547656,0.627070,0.661300,0.534155,0.627165,0.685404,0.762597,0.747642,0.620451,0.697903,0.672400,0.590494,0.548872,0.698895,0.725543,0.568075,0.666121,0.551495,0.580087,0.400901,0.580282,0.634049,0.671031,0.649669,0.635404,0.685451,0.577523,0.677707,0.654135,0.740741,0.673945,0.669828,0.680702,0.772358,0.618824,0.719156,0.685466,0.632450,0.388953,0.793224,0.366226,0.785356,0.799818,0.687636
Blanco,0.002262,0.008606,0.008466,0.005351,0.002817,0.005089,0.005964,0.010135,0.001316,0.003700,0.006024,0.007329,0.007787,0.010995,0.005922,0.004956,0.005521,0.003781,0.001967,0.001416,0.006061,0.002283,0.003125,0.002160,0.003450,0.008959,0.004329,0.001972,0.010659,0.004717,0.008666,0.011796,0.002346,0.000000,0.007519,0.006906,0.004076,0.003756,0.003273,0.001661,0.002165,0.001802,0.000000,0.003994,0.009002,0.002833,0.003517,0.011270,0.003281,0.005096,0.001504,0.001852,0.002911,0.006897,0.004678,0.005807,0.002353,0.008117,0.001085,0.003311,0.005754,0.003505,0.003052,0.003009,0.002730,0.002169
V_Cand,0.769231,0.663224,0.656085,0.685493,0.656338,0.642918,0.618953,0.729730,0.702632,0.595745,0.666667,0.748371,0.727450,0

In [246]:
secciones_select_norm.shape

(30, 66)

Ya podemos aplucar el modelo, y para ello definimos la matriz X e y.

In [247]:
secciones_select_norm['Modelización'] = modelizacion['Modelización']

In [248]:
X = secciones_select_norm.drop('Modelización', axis = 1).values
y = secciones_select_norm['Modelización'].values

Aplicamos el modelo del pipeline, por supuesto sin hacer ningún fit.

In [249]:
predicciones = pipe_modelado.predict(X=X)
predicciones = predicciones.flatten()

In [250]:
r2 = r2_score(
            y_true  = y,
            y_pred  = predicciones
           )

El R2 que conseguimos es muy bueno, un 99.97%

In [251]:
r2

0.9997383554666492

Ahora mostramos los resultados, comenzando por deshacer la normalización.

In [252]:
est = predicciones * censo_mod

In [253]:
df = pd.DataFrame(est, index = secciones_select_norm.index, columns = ['Estimación']).astype('int32')

In [254]:
df

,Estimación
Votos_Total,24400768
Nulos,238687
Votos_Válidos,24161386
Blanco,162773
V_Cand,23997919
PP,7577739
PSOE,5585833
Cs,3176265
UP,5334831
IU,-694


Añadimos la columna de los datos reales en 2016

In [255]:
df1 = pd.DataFrame(secciones_mod.sum(), columns = ['Real']).drop('Censo_Esc')

In [256]:
df['Real'] = df1['Real']

In [257]:
df

,Estimación,Real
Votos_Total,24400768,24157982
Nulos,238687,224564
Votos_Válidos,24161386,23933418
Blanco,162773,178559
V_Cand,23997919,23754859
PP,7577739,7906761
PSOE,5585833,5424130
Cs,3176265,3123436
UP,5334831,5051345
IU,-694,0


Añadimos columnas con los porcentajes de voto para majorar la interpretabilidad. 

In [258]:
df['pc Estimación'] = df['Estimación'] / df['Estimación'][2] * 100

In [259]:
df['pc Real'] = df['Real'] / df['Real'][2] * 100

In [260]:
df['dif. Real-Est.'] = df['pc Real'] - df['pc Estimación']

In [261]:
df

,Estimación,Real,pc Estimación,pc Real,dif. Real-Est.
Votos_Total,24400768,24157982,100.990763,100.938286,-0.052476
Nulos,238687,224564,0.987886,0.938286,-0.049600
Votos_Válidos,24161386,23933418,100.000000,100.000000,0.000000
Blanco,162773,178559,0.673691,0.746066,0.072375
V_Cand,23997919,23754859,99.323437,99.253934,-0.069503
PP,7577739,7906761,31.363015,33.036489,1.673474
PSOE,5585833,5424130,23.118843,22.663416,-0.455428
Cs,3176265,3123436,13.146038,13.050522,-0.095516
UP,5334831,5051345,22.079987,21.105824,-0.974164
IU,-694,0,-0.002872,0.000000,0.002872


Vemos que el ajuste no es igual de bueno que en noviembre de 2019, como era de esperar, pero sigue siendo bastante satisfactorio en general. La estimación solo difiere de la realidad un poco en el caso del PP.

Pensamos que tiene bastante mérito, teniendo en cuenta la forma casi minimalista en la que hemos elegido las fuentes de las secciones seleccionadas: una serie de municipios elegidos al tuntun en provincias en general con partidos nacionalistas.